In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

#import lightgbm as lgb
import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

# Preprocessing

In [5]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [6]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [7]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [8]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=train[scale_features], y=train[target]):
  """
  """
  param_space = {
               'objective': 'reg:squarederror',
               'learning_rate':trial.suggest_uniform('learning_rate', 0.65e-3, 2.65e-3),
               'n_estimators':N_ESTIMATORS,
               'subsample': trial.suggest_uniform('subsample', 0.52, 0.62),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.704, 0.904),
              'max_depth':trial.suggest_int('max_depth', 11, 15),
              'lambda':trial.suggest_uniform('lambda', 6.955, 11.955),
              'alpha':trial.suggest_uniform('alpha', 3.979, 8.979),
           'min_child_weight':trial.suggest_uniform('min_child_weight', 116, 216),
           'importance_type': 'total_gain',
            'tree_method': 'hist'
               # 'subsample_freq': trial.suggest_int('subsample_freq', 1, 5),
             #'boosting_type': 'gbdt',
               
                # 'min_child_samples':trial.suggest_int('min_child_samples', 5, 35),
             # 'num_leaves':trial.suggest_int('num_leaves', 10, 200),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
              # 'n_jobs' : -1,
                #'max_bin':trial.suggest_int('max_bin', 300, 1000),
              #'cat_smooth':trial.suggest_int('cat_smooth', 5, 100),
              #'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 100)
                }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  
  pred=np.zeros(y_va.shape[0])
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  
  #X_tr = apply_noise_row(X_tr)

  
  
  #start = time.time()
  for inseed in seed_list:
            
            param_space['seed'] = inseed

            model = xgb.XGBRegressor(**param_space)
            model.fit(
                X_tr, 
                y_tr,
                eval_set=[(X_va, y_va)],
                eval_metric='rmse',
                early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                verbose=VERBOSE,
            )


            pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [9]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-22 10:57:54,559] A new study created in memory with name: no-name-b55e377a-4cc6-434d-82a0-1bb93914ea64


[0]	validation_0-rmse:10.05525


[1000]	validation_0-rmse:8.16670


[2000]	validation_0-rmse:7.84509


[3000]	validation_0-rmse:7.78953


...

[6998]	validation_0-rmse:7.75455


[I 2021-08-22 19:49:23,680] Trial 10 finished with value: 7.754114367996892 and parameters: {'learning_rate': 0.0017611202036189068, 'subsample': 0.6192014522206959, 'colsample_bytree': 0.9017795656542079, 'max_depth': 13, 'lambda': 10.33480455799122, 'alpha': 5.644892485497893, 'min_child_weight': 215.39670069031698}. Best is trial 4 with value: 7.753198149027122.


[0]	validation_0-rmse:10.05219


[1000]	validation_0-rmse:7.87856


[2000]	validation_0-rmse:7.78132


In [ ]:
study.best_params

# Log

7.76776099796762 random noise ver3

7.767604136668598 row-wise noise ver4

7.753515220316349 no noise ver5

7.7532905240188965 no noise ver6 (narrow space)

kfold row-wise noise ver2
7.847220305678095